In [1]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()
api.competition_download_file('time-series-classification-part-1','ETTh1_without_missing.csv')

100%|██████████| 753k/753k [00:00<00:00, 1.36MB/s]

In [2]:
import csv
def get_data_from_csv(path):
    row = []
    with open(path, 'r') as file:
        i = 0
        reader = csv.reader(file, delimiter=';')
        data = {}
        for row in reader:
            if(i != 0):
                row_values = {}
                row_values["date"] = (row[0].split(","))[1].split(" ")[0]
                row_values["hour"] = int((row[0].split(","))[1].split(" ")[1].split(":")[0])
                row_values["OT"] = float((row[0].split(","))[2])
                id = int(row[0].split(",")[0])
                data[id] = row_values
            i += 1
        return data

data = get_data_from_csv('ETTh1_without_missing.csv')
print(data[0])



{'date': '2016-07-01', 'hour': 0, 'OT': 30.5310001373291}


In [3]:
from datetime import datetime, timedelta

# Get the last value to predict the next 100 values
last_entry = data[max(data.keys())]

# Extract the date and hour from the last value
last_date = datetime.strptime(last_entry['date'], "%Y-%m-%d")  # Convert the date string to a datetime object
last_hour = last_entry['hour']

next_100_dates = []  # List to store the next 100 dates

# Generate the next 100 dates
for _ in range(100):
    last_hour += 1  # Increment the hour by 1
    next_date = last_date + timedelta(hours=last_hour)  # Calculate the next date

    # Format the next date to the desired string format
    next_date_formatted = next_date.strftime("%Y-%m-%d %H:%M:%S")
    next_100_dates.append(next_date_formatted)  # Append the formatted next date to the list

# The list next_100_dates now contains the next 100 dates
print(next_100_dates[0])


2018-06-22 16:00:00


In [9]:
# extract data to learn on
extracted_data = {cle: valeur for i, (cle, valeur) in enumerate(data.items()) if i >= len(data)-1000}
print(len(extracted_data))

1000


In [5]:
from sklearn.model_selection import train_test_split
from datetime import datetime
import numpy as np
import tensorflow as tf

# Extract the dates from the data by converting strings to datetime objects
dates = [datetime.strptime(value['date'], '%Y-%m-%d') for _, value in extracted_data.items()]

# Extract the years, months, days, and hours from the datetime objects
years = [date.year for date in dates]
months = [date.month for date in dates]
days = [date.day for date in dates]
hours = [value['hour'] for _, value in extracted_data.items()]

# Extract the targets (OT values) from the data
targets = [value['OT'] for _, value in extracted_data.items()]

# Stack the years, months, days, and hours into a numpy array
input = np.column_stack((years, months, days, hours))

# Split the data into training and testing sets
# 80% of the data will be used for training and 20% for testing
# Data is split randomly, with random_state=42 to ensure reproducibility
X_train, X_test, y_train, y_test = train_test_split(input, targets, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(np.array(X_train), np.array(y_train), test_size=0.15, random_state=42)

# Create TensorFlow datasets from arrays
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

batch_size_value = 32
# Shuffle and batch the training data
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size_value)

# Batch the validation and test data
val_dataset = val_dataset.batch(batch_size_value)
test_dataset = test_dataset.batch(batch_size_value)

# Display set lengths
print("Train set length :", len(X_train), "samples")
print("Validation set length :", len(X_val), "samples")
print("Test set length :", len(X_test), "samples")
print("Total :", len(X_train)+len(X_test)+len(X_val), "samples")

c:\Users\Nassim\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Train set length : 722 samples
Validation set length : 128 samples
Test set length : 150 samples
Total : 1000 samples


In [6]:
import tensorflow as tf

# Model creation
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=1)  
])

# Compilation
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Training
history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size_value, validation_data=(X_val, np.array(y_val)))

# Evaluation
loss, mae = model.evaluate(X_test, np.array(y_test))
print("Test loss :", loss)
print("Test MAE :", mae)




Epoch 1/50


23/23 [==============================] - 3s 20ms/step - loss: 156.1490 - mae: 9.5836 - val_loss: 24.8496 - val_mae: 4.4838
Epoch 2/50
23/23 [==============================] - 0s 5ms/step - loss: 12.9101 - mae: 2.9291 - val_loss: 13.3419 - val_mae: 3.1771
Epoch 3/50
23/23 [==============================] - 0s 5ms/step - loss: 7.5081 - mae: 2.1515 - val_loss: 6.9161 - val_mae: 1.8961
Epoch 4/50
23/23 [==============================] - 0s 5ms/step - loss: 6.0383 - mae: 1.8654 - val_loss: 6.2525 - val_mae: 1.7637
Epoch 5/50
23/23 [==============================] - 0s 5ms/step - loss: 6.5549 - mae: 1.9750 - val_loss: 6.5074 - val_mae: 1.8142
Epoch 6/50
23/23 [==============================] - 0s 5ms/step - loss: 6.1483 - mae: 1.8943 - val_loss: 6.1444 - val_mae: 1.8572
Epoch 7/50
23/23 [==============================] - 0s 5ms/step - loss: 5.7477 - mae: 1.8246 - val_loss: 6.0888 - val_mae: 1.8416
Epoch 8/50
23/23 [==============================] - 0s 6ms/step - loss: 5.8965 -

In [7]:
def preprocess_next_date(date, hour):
    
    years = int(date.split("-")[0])
    months = int(date.split("-")[1])
    days = int(date.split("-")[2])

    return np.array([[years, months, days, hour]])

In [8]:
def predict_next_100_values(filename):
    fields = ["Id", "OT"]

    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(fields)
        
        for i in range(100):
            csv_line = []
            date = next_100_dates[i].split(" ")[0]
            hour = int(next_100_dates[i].split(" ")[1].split(":")[0])
            new_value_prediction = model.predict(preprocess_next_date(date, hour))
            csv_line = [i, float(new_value_prediction)]
            writer.writerow(csv_line)
    print(f"CSV file '{filename}' successfully created.")

filename = "neural_network_prediction.csv"
predict_next_100_values(filename)

1/1 [==============================] - 0s 37ms/step
CSV file 'neural_network_prediction.csv' successfully created.
